In [1]:
import os
import time
import pandas as pd
import numpy as np

# DATA_FILEPATH     = '/Volumes/LaCie/datasets/Physionet/mimic_extract/all_hourly_data.h5'
DATA_FILEPATH     = './data/mimic/all_hourly_data.h5'

GAP_TIME          = 6  # In hours
WINDOW_SIZE       = 24 # In hours
SEED              = 1
ID_COLS           = ['subject_id', 'hadm_id', 'icustay_id']

np.random.seed(SEED)

data_full_lvl2 = pd.read_hdf(DATA_FILEPATH, 'vitals_labs')
statics        = pd.read_hdf(DATA_FILEPATH, 'patients')

In [2]:
data_full_lvl2.head()

LEVEL2                                 alanine aminotransferase             \
Aggregation Function                                      count  mean  std   
subject_id hadm_id icustay_id hours_in                                       
3          145834  211552     0                             2.0  25.0  0.0   
                              1                             0.0   NaN  NaN   
                              2                             0.0   NaN  NaN   
                              3                             0.0   NaN  NaN   
                              4                             0.0   NaN  NaN   

LEVEL2                                 albumin           albumin ascites       \
Aggregation Function                     count mean  std           count mean   
subject_id hadm_id icustay_id hours_in                                          
3          145834  211552     0            2.0  1.8  0.0             0.0  NaN   
                              1            0.0  NaN  NaN             0.0  NaN   
                              2            0.0  NaN  NaN             0.0  NaN   
                              3            0.0  NaN  NaN             0.0  NaN   
                              4            0.0  NaN  NaN             0.0  NaN   

LEVEL2                                     albumin pleural  ...  \
Aggregation Function                   std           count  ...   
subject_id hadm_id icustay_id hours_in                      ...   
3          145834  211552     0        NaN             0.0  ...   
                              1        NaN             0.0  ...   
                              2        NaN             0.0  ...   
                              3        NaN             0.0  ...   
                              4        NaN             0.0  ...   

LEVEL2                                 white blood cell count  \
Aggregation Function                                      std   
subject_id hadm_id icustay_id hours_in                          
3          145834  211552     0                      4.012837   
                              1                           NaN   
                              2                           NaN   
                              3                           NaN   
                              4                           NaN   

LEVEL2                                 white blood cell count urine           \
Aggregation Function                                          count mean std   
subject_id hadm_id icustay_id hours_in                                         
3          145834  211552     0                                 0.0  NaN NaN   
                              1                                 0.0  NaN NaN   
                              2                                 0.0  NaN NaN   
                              3                                 0.0  NaN NaN   
                              4                                 0.0  NaN NaN   

LEVEL2                                    ph                 ph urine           
Aggregation Function                   count  mean       std    count mean std  
subject_id hadm_id icustay_id hours_in                                          
3          145834  211552     0          9.0  7.40  0.147733      1.0  5.0 NaN  
                              1          0.0   NaN       NaN      0.0  NaN NaN  
                              2          3.0  7.26  0.000000      0.0  NaN NaN  
                              3          0.0   NaN       NaN      0.0  NaN NaN  
                              4          0.0   NaN       NaN      0.0  NaN NaN  

[5 rows x 312 columns]

In [3]:
len(set(data_full_lvl2.columns.get_level_values(0)))

104

In [4]:
statics.head()

,,,gender,ethnicity,age,insurance,admittime,diagnosis_at_admission,dischtime,discharge_location,fullcode_first,dnr_first,...,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,readmission_30,max_hours
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,,,,
3,145834,211552,M,WHITE,76.526792,Medicare,2101-10-20 19:08:00,HYPOTENSION,2101-10-31 13:58:00,SNF,1.0,0.0,...,2101-10-26 20:43:09,6.064560,EMERGENCY,MICU,0,0,0,1,0,145
4,185777,294638,F,WHITE,47.845047,Private,2191-03-16 00:28:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",2191-03-23 18:41:00,HOME WITH HOME IV PROVIDR,1.0,0.0,...,2191-03-17 16:46:31,1.678472,EMERGENCY,MICU,0,0,0,1,0,40
6,107064,228232,F,WHITE,65.942297,Medicare,2175-05-30 07:15:00,CHRONIC RENAL FAILURE/SDA,2175-06-15 16:00:00,HOME HEALTH CARE,1.0,0.0,...,2175-06-03 13:39:54,3.672917,ELECTIVE,SICU,0,0,0,1,0,88
9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.790228,Medicaid,2149-11-09 13:06:00,HEMORRHAGIC CVA,2149-11-14 10:15:00,DEAD/EXPIRED,1.0,0.0,...,2149-11-14 20:52:14,5.323056,EMERGENCY,MICU,1,1,1,1,0,127
11,194540,229441,F,WHITE,50.148295,Private,2178-04-16 06:18:00,BRAIN MASS,2178-05-11 19:00:00,HOME HEALTH CARE,1.0,0.0,...,2178-04-17 20:21:05,1.584410,EMERGENCY,SICU,0,0,0,1,0,38


In [5]:
def simple_imputer(df):
    idx = pd.IndexSlice
    df = df.copy()
    if len(df.columns.names) > 2: df.columns = df.columns.droplevel(('label', 'LEVEL1', 'LEVEL2'))
    
    df_out = df.loc[:, idx[:, ['mean', 'count']]]
    icustay_means = df_out.loc[:, idx[:, 'mean']].groupby(ID_COLS).mean()
    
    df_out.loc[:,idx[:,'mean']] = df_out.loc[:,idx[:,'mean']].groupby(ID_COLS).fillna(
        method='ffill'
    ).groupby(ID_COLS).fillna(icustay_means).fillna(0)
    
    df_out.loc[:, idx[:, 'count']] = (df.loc[:, idx[:, 'count']] > 0).astype(float)
    df_out.rename(columns={'count': 'mask'}, level='Aggregation Function', inplace=True)
    
    is_absent = (1 - df_out.loc[:, idx[:, 'mask']])
    hours_of_absence = is_absent.cumsum()
    time_since_measured = hours_of_absence - hours_of_absence[is_absent==0].fillna(method='ffill')
    time_since_measured.rename(columns={'mask': 'time_since_measured'}, level='Aggregation Function', inplace=True)

    df_out = pd.concat((df_out, time_since_measured), axis=1)
    df_out.loc[:, idx[:, 'time_since_measured']] = df_out.loc[:, idx[:, 'time_since_measured']].fillna(100)
    
    df_out.sort_index(axis=1, inplace=True)
    return df_out

In [6]:
# This was not mentioned in Suriyakumar paper: extra preprocess to match cohort selection in Suriyakumar paper
# drop the NaN for diagnosis_at_admission (1 person)
statics = statics[statics['diagnosis_at_admission'].notnull()]
# filter rows to los_icu > 36 hours
statics = statics[statics['los_icu'] > 1.5]
statics['los_icu'].describe()

count    21877.000000
mean         3.554777
std          1.967715
min          1.500046
25%          2.048958
50%          2.907998
75%          4.391910
max          9.999456
Name: los_icu, dtype: float64

In [7]:
Ys = statics[statics.max_hours > WINDOW_SIZE + GAP_TIME][['mort_hosp', 'mort_icu', 'los_icu']]
Ys['los_3'] = Ys['los_icu'] > 3
Ys['los_7'] = Ys['los_icu'] > 7
# Ys.drop(columns=['los_icu'], inplace=True)
Ys.astype(float)

df = data_full_lvl2
lvl2 = df[
    (df.index.get_level_values('icustay_id').isin(set(Ys.index.get_level_values('icustay_id')))) &
    (df.index.get_level_values('hours_in') < WINDOW_SIZE) ]  

train_frac, dev_frac, test_frac = 0.7, 0.1, 0.2
lvl2_subj_idx, Ys_subj_idx = [df.index.get_level_values('subject_id') for df in (lvl2, Ys)]
lvl2_subjects = set(lvl2_subj_idx)
assert lvl2_subjects == set(Ys_subj_idx), "Subject ID pools differ!"

# shuffle the dataset
np.random.seed(SEED)
subjects, N = np.random.permutation(list(lvl2_subjects)), len(lvl2_subjects)
N_train, N_dev, N_test = int(train_frac * N), int(dev_frac * N), int(test_frac * N)
train_subj = subjects[:N_train]
dev_subj   = subjects[N_train:N_train + N_dev]
test_subj  = subjects[N_train+N_dev:]

[(lvl2_train, lvl2_dev, lvl2_test), (Ys_train, Ys_dev, Ys_test)] = [
    [df[df.index.get_level_values('subject_id').isin(s)] for s in (train_subj, dev_subj, test_subj)] \
    for df in (lvl2,  Ys)
]


In [8]:
## Save targets: 
# Ys.to_csv('data/mimic/Ys_mimic_extract.csv')
Ys_train.to_csv('data/mimic/Ys_train.csv')
Ys_dev.to_csv('data/mimic/Ys_dev.csv')
Ys_test.to_csv('data/mimic/Ys_test.csv')

## read in the cohort selection
# Ysr = pd.read_csv('data/mimic/Ys_mimic_extract.csv', index_col=[0,1,2])
# Ysr.head()

In [9]:
# normalize the dataset w/ z-score
idx = pd.IndexSlice
lvl2_means, lvl2_stds = lvl2_train.loc[:, idx[:,'mean']].mean(axis=0), lvl2_train.loc[:, idx[:,'mean']].std(axis=0)

dfz_train, dfz_dev, dfz_test = lvl2_train.copy(), lvl2_dev.copy(), lvl2_test.copy()

dfz_train.loc[:, idx[:,'mean']] = (dfz_train.loc[:, idx[:,'mean']] - lvl2_means)/lvl2_stds
dfz_dev.loc[:, idx[:,'mean']] = (dfz_dev.loc[:, idx[:,'mean']] - lvl2_means)/lvl2_stds
dfz_test.loc[:, idx[:,'mean']] = (dfz_test.loc[:, idx[:,'mean']] - lvl2_means)/lvl2_stds

In [10]:
# l infinity normalization of the dataset
lvl2_absmax = lvl2_train.loc[:, idx[:,'mean']].abs().max(axis=0)


In [11]:

df_train, df_dev, df_test = lvl2_train.copy(), lvl2_dev.copy(), lvl2_test.copy()

df_train.loc[:, idx[:,'mean']] = (df_train.loc[:, idx[:,'mean']] )/lvl2_absmax
df_dev.loc[:, idx[:,'mean']] = (df_dev.loc[:, idx[:,'mean']] )/lvl2_absmax
df_test.loc[:, idx[:,'mean']] = (df_test.loc[:, idx[:,'mean']] )/lvl2_absmax

In [12]:
#compute rate of missingness 
df = pd.concat([lvl2_train, lvl2_dev, lvl2_test], axis = 0)
df = df.loc[:, idx[:, 'mean']]
print(f' rate of missingness before imputation is {df.isna().sum().sum()} / {df.shape[0] * df.shape[1]} = {df.isna().sum().sum() / (df.shape[0] * df.shape[1])}')

 rate of missingness before imputation is 48650661 / 54604992 = 0.8909562883920943


In [13]:
df_train.loc[:, idx[:, 'mean']].head()

LEVEL2                                 alanine aminotransferase   albumin  \
Aggregation Function                                       mean      mean   
subject_id hadm_id icustay_id hours_in                                      
3          145834  211552     0                          0.0025  0.321429   
                              1                             NaN       NaN   
                              2                             NaN       NaN   
                              3                             NaN       NaN   
                              4                             NaN       NaN   

LEVEL2                                 albumin ascites albumin pleural  \
Aggregation Function                              mean            mean   
subject_id hadm_id icustay_id hours_in                                   
3          145834  211552     0                    NaN             NaN   
                              1                    NaN             NaN   
                              2                    NaN             NaN   
                              3                    NaN             NaN   
                              4                    NaN             NaN   

LEVEL2                                 albumin urine alkaline phosphate  \
Aggregation Function                            mean               mean   
subject_id hadm_id icustay_id hours_in                                    
3          145834  211552     0                  NaN           0.026927   
                              1                  NaN                NaN   
                              2                  NaN                NaN   
                              3                  NaN                NaN   
                              4                  NaN                NaN   

LEVEL2                                 anion gap asparate aminotransferase  \
Aggregation Function                        mean                      mean   
subject_id hadm_id icustay_id hours_in                                       
3          145834  211552     0         0.413333                   0.00345   
                              1              NaN                       NaN   
                              2              NaN                       NaN   
                              3              NaN                       NaN   
                              4              NaN                       NaN   

LEVEL2                                 basophils bicarbonate  ...  \
Aggregation Function                        mean        mean  ...   
subject_id hadm_id icustay_id hours_in                        ...   
3          145834  211552     0              NaN    0.308176  ...   
                              1              NaN         NaN  ...   
                              2              NaN         NaN  ...   
                              3              NaN         NaN  ...   
                              4              NaN         NaN  ...   

LEVEL2                                 total protein total protein urine  \
Aggregation Function                            mean                mean   
subject_id hadm_id icustay_id hours_in                                     
3          145834  211552     0                  NaN                 NaN   
                              1                  NaN                 NaN   
                              2                  NaN                 NaN   
                              3                  NaN                 NaN   
                              4                  NaN                 NaN   

LEVEL2                                 troponin-i troponin-t venous pvo2  \
Aggregation Function                         mean       mean        mean   
subject_id hadm_id icustay_id hours_in                                     
3          145834  211552     0               NaN        NaN         NaN   
                              1               NaN        NaN         NaN   
                      

In [14]:
# impute missing values
df_train, df_dev, df_test, dfz_train, dfz_dev, dfz_test = [
    simple_imputer(df) for df in (df_train, df_dev, df_test, dfz_train, dfz_dev, dfz_test)
    ]


/var/folders/fy/kszc5b_n1dn3c5g8vg11ck4nvtb7xh/T/ipykernel_90884/2355116716.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out.loc[:,idx[:,'mean']] = df_out.loc[:,idx[:,'mean']].groupby(ID_COLS).fillna(
/var/folders/fy/kszc5b_n1dn3c5g8vg11ck4nvtb7xh/T/ipykernel_90884/2355116716.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out.loc[:, idx[:, 'count']] = (df.loc[:, idx[:, 'count']] > 0).astype(float)
/var/folders/fy/kszc5b_n1dn3c5g8vg11ck4nvtb7xh/T/ipykernel_90884/2355116716.py:14: SettingWi

In [15]:
for df in df_train, df_dev, df_test: assert not df.isnull().any().any()

### save preprocess data as h5 file

In [16]:
# save the data at this point.
# in practice we'll probably want to pivot it after reading it in. 
# for GRU-D likely multiply the mask by the mean (killing off the imputed values) 
# likely will ignore the time_since columns. 

dfz_train.to_hdf('data/mimic/lvl2_z_normalized.h5', key='train')
dfz_dev.to_hdf('data/mimic/lvl2_z_normalized.h5', key='dev')
dfz_test.to_hdf('data/mimic/lvl2_z_normalized.h5', key='test')

df_train.to_hdf('data/mimic/lvl2_l_inf_normalized.h5', key='train')
df_dev.to_hdf('data/mimic/lvl2_l_inf_normalized.h5', key='dev')
df_test.to_hdf('data/mimic/lvl2_l_inf_normalized.h5', key='test')



/Users/r8t/ppo/privacy_preserving_optimization/.venv/lib/python3.9/site-packages/tables/attributeset.py:459: NaturalNameWarning: object name is not a valid Python identifier: 'axis0_nameAggregation Function'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/Users/r8t/ppo/privacy_preserving_optimization/.venv/lib/python3.9/site-packages/tables/attributeset.py:459: NaturalNameWarning: object name is not a valid Python identifier: 'block0_items_nameAggregation Function'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


## Examples of using saved data for ML: 

In [56]:
with pd.HDFStore('data/mimic/lvl2_l_inf_normalized.h5') as hdf:
    print(hdf.keys())
    print(hdf['train'].shape)
    
# re-load data example: 
df_train2 = pd.read_hdf('data/mimic/lvl2_l_inf_normalized.h5', key = 'train')
df_train2.head()

['/dev', '/test', '/train']
(367512, 312)


LEVEL2                                 alanine aminotransferase          \
Aggregation Function                                       mask    mean   
subject_id hadm_id icustay_id hours_in                                    
3          145834  211552     0                             1.0  0.0025   
                              1                             0.0  0.0025   
                              2                             0.0  0.0025   
                              3                             0.0  0.0025   
                              4                             0.0  0.0025   

LEVEL2                                                     albumin            \
Aggregation Function                   time_since_measured    mask      mean   
subject_id hadm_id icustay_id hours_in                                         
3          145834  211552     0                        0.0     1.0  0.321429   
                              1                        1.0     0.0  0.321429   
                              2                        2.0     0.0  0.321429   
                              3                        3.0     0.0  0.321429   
                              4                        4.0     0.0  0.321429   

LEVEL2                                                     albumin ascites  \
Aggregation Function                   time_since_measured            mask   
subject_id hadm_id icustay_id hours_in                                       
3          145834  211552     0                        0.0             0.0   
                              1                        1.0             0.0   
                              2                        2.0             0.0   
                              3                        3.0             0.0   
                              4                        4.0             0.0   

LEVEL2                                                           \
Aggregation Function                   mean time_since_measured   
subject_id hadm_id icustay_id hours_in                            
3          145834  211552     0         0.0               100.0   
                              1         0.0               100.0   
                              2         0.0               100.0   
                              3         0.0               100.0   
                              4         0.0               100.0   

LEVEL2                                 albumin pleural  ...  \
Aggregation Function                              mask  ...   
subject_id hadm_id icustay_id hours_in                  ...   
3          145834  211552     0                    0.0  ...   
                              1                    0.0  ...   
                              2                    0.0  ...   
                              3                    0.0  ...   
                              4                    0.0  ...   

LEVEL2                                         venous pvo2 weight         \
Aggregation Function                   time_since_measured   mask   mean   
subject_id hadm_id icustay_id hours_in                                     
3          145834  211552     0                      100.0    0.0  0.424   
                              1                      100.0    0.0  0.424   
                              2                      100.0    0.0  0.424   
                              3                      100.0    0.0  0.424   
                              4                      100.0    0.0  0.424   

LEVEL2                                                      \
Aggregation Function                   time_since_measured   
subject_id hadm_id icustay_id hours_in                       
3          145834  211552     0                      100.0   
                              1                      100.0   
                              2                      100.0   
                              3                      100.0   
                              4                 

In [45]:
# pivot on the hourly measurements to make three levels
df3_train, df3_dev, df3_test = [
    df.pivot_table(index=['subject_id', 'hadm_id', 'icustay_id'], columns=['hours_in']) \
        for df in (df_train, df_dev, df_test )]
df3_train.head()

LEVEL2                        alanine aminotransferase                      \
Aggregation Function                              mask                       
hours_in                                            0    1    2    3    4    
subject_id hadm_id icustay_id                                                
3          145834  211552                          1.0  0.0  0.0  0.0  0.0   
6          107064  228232                          1.0  0.0  0.0  0.0  0.0   
9          150750  220597                          0.0  0.0  0.0  0.0  0.0   
11         194540  229441                          0.0  0.0  0.0  0.0  0.0   
12         112213  232669                          0.0  0.0  0.0  0.0  0.0   

LEVEL2                                                  ...  \
Aggregation Function                                    ...   
hours_in                        5    6    7    8    9   ...   
subject_id hadm_id icustay_id                           ...   
3          145834  211552      0.0  0.0  0.0  0.0  0.0  ...   
6          107064  228232      0.0  0.0  0.0  0.0  0.0  ...   
9          150750  220597      0.0  0.0  0.0  0.0  0.0  ...   
11         194540  229441      0.0  0.0  0.0  0.0  0.0  ...   
12         112213  232669      0.0  0.0  0.0  0.0  0.0  ...   

LEVEL2                        white blood cell count urine                \
Aggregation Function                   time_since_measured                 
hours_in                                                14     15     16   
subject_id hadm_id icustay_id                                              
3          145834  211552                              6.0    7.0    8.0   
6          107064  228232                             30.0   31.0   32.0   
9          150750  220597                             54.0   55.0   56.0   
11         194540  229441                             78.0   79.0   80.0   
12         112213  232669                            102.0  103.0  104.0   

LEVEL2                                                                          
Aggregation Function                                                            
hours_in                          17     18     19     20     21     22     23  
subject_id hadm_id icustay_id                                                   
3          145834  211552        9.0   10.0   11.0   12.0   13.0   14.0   15.0  
6          107064  228232       33.0   34.0   35.0   36.0   37.0   38.0   39.0  
9          150750  220597       57.0   58.0   59.0   60.0   61.0   62.0   63.0  
11         194540  229441       81.0   82.0   83.0   84.0   85.0   86.0   87.0  
12         112213  232669      105.0  106.0  107.0  108.0  109.0  110.0  111.0  

[5 rows x 7488 columns]

In [57]:
df3_train.loc[:, idx[:, 'mean']].head()

LEVEL2                        alanine aminotransferase                  \
Aggregation Function                              mean                   
hours_in                                            0       1       2    
subject_id hadm_id icustay_id                                            
3          145834  211552                       0.0025  0.0025  0.0025   
6          107064  228232                       0.0023  0.0023  0.0023   
9          150750  220597                       0.0011  0.0011  0.0011   
11         194540  229441                       0.0000  0.0000  0.0000   
12         112213  232669                       0.0000  0.0000  0.0000   

LEVEL2                                                                         \
Aggregation Function                                                            
hours_in                           3       4       5       6       7       8    
subject_id hadm_id icustay_id                                                   
3          145834  211552      0.0025  0.0025  0.0025  0.0025  0.0025  0.0025   
6          107064  228232      0.0023  0.0023  0.0023  0.0023  0.0023  0.0023   
9          150750  220597      0.0011  0.0011  0.0011  0.0011  0.0011  0.0011   
11         194540  229441      0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   
12         112213  232669      0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   

LEVEL2                                 ... white blood cell count urine  \
Aggregation Function                   ...                         mean   
hours_in                           9   ...                           14   
subject_id hadm_id icustay_id          ...                                
3          145834  211552      0.0025  ...                     0.040046   
6          107064  228232      0.0023  ...                     0.000000   
9          150750  220597      0.0011  ...                     0.000000   
11         194540  229441      0.0000  ...                     0.000000   
12         112213  232669      0.0000  ...                     0.000000   

LEVEL2                                                                 \
Aggregation Function                                                    
hours_in                             15        16        17        18   
subject_id hadm_id icustay_id                                           
3          145834  211552      0.040046  0.040046  0.040046  0.040046   
6          107064  228232      0.000000  0.000000  0.000000  0.000000   
9          150750  220597      0.000000  0.000000  0.000000  0.000000   
11         194540  229441      0.000000  0.000000  0.000000  0.000000   
12         112213  232669      0.000000  0.000000  0.000000  0.000000   

LEVEL2                                                                 \
Aggregation Function                                                    
hours_in                             19        20        21        22   
subject_id hadm_id icustay_id                                           
3          145834  211552      0.040046  0.040046  0.040046  0.040046   
6          107064  228232      0.000000  0.000000  0.000000  0.000000   
9          150750  220597      0.000000  0.000000  0.000000  0.000000   
11         194540  229441      0.000000  0.000000  0.000000  0.000000   
12         112213  232669      0.000000  0.000000  0.000000  0.000000   

LEVEL2                                   
Aggregation Function                     
hours_in                             23  
subject_id hadm_id icustay_id            
3          145834  211552      0.040046  
6          107064  228232      0.000000  
9          150750  220597      0.000000  
11         194540  229441      0.000000  
12         112213  232669      0.000000  

[5 rows x 2496 columns]

In [59]:
# CS, check shapes

print(df_train.shape)
print(df_dev.shape)
print(df_test.shape)
print()
print(df3_train.shape)
print(df3_dev.shape)
print(df3_test.shape)
print()
print(367512 * 312)
print(15313 * 7488)

(367512, 312)
(52488, 312)
(105048, 312)

(15313, 7488)
(2187, 7488)
(4377, 7488)

114663744
114663744
